## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
...,...,...
1465,No,Research & Development
1466,No,Research & Development
1467,No,Research & Development
1468,No,Sales


In [5]:
# Create a list of at least 10 column names to use as X data
column_names = ['Education', 'Age','DistanceFromHome', 'JobSatisfaction', 
                'OverTime', 'StockOptionLevel', 'WorkLifeBalance',
                'YearsAtCompany', 'YearsSinceLastPromotion', 
                'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[column_names]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)

In [7]:
X_train['OverTime'].value_counts()

OverTime
No     780
Yes    322
Name: count, dtype: int64

In [8]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
from sklearn.preprocessing import LabelEncoder

overTime_encoder = LabelEncoder()
X_train['OverTime'] = overTime_encoder.fit_transform(X_train['OverTime'])
X_test['OverTime'] = overTime_encoder.fit_transform(X_test['OverTime'])

X_train['OverTime'].value_counts()

OverTime
0    780
1    322
Name: count, dtype: int64

In [35]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
#scaler.fit(X_train, y_train)

# Scale the training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)
overtime_columns = scaler.get_feature_names_out(['OverTime'])
df_overtime_scaled = pd.DataFrame(X_train_scaled, columns=overtime_columns)
X_df = pd.concat([X_df, df_overtime_scaled], axis=1)
X_df = X_df.drop(['OverTime'], axis=1)
X_df.head()

#X_train_scaled

ValueError: input_features is not equal to feature_names_in_

In [34]:
df_overtime_scaled.head()

NameError: name 'df_overtime_scaled' is not defined

In [26]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder

# Fit the encoder to the training data
department_encoder = OneHotEncoder(sparse_output=False)

# Create two new variables by applying the encoder
# to the training and testing data
department_training_encoded = department_encoder.fit_transform(y_train[['Department']])
department_testing_encoded = department_encoder.fit_transform(y_test[['Department']])
department_columns = department_encoder.get_feature_names_out(['Department'])
df_department = pd.DataFrame(department_training_encoded, columns=department_columns)
y_df = pd.concat([y_df, df_department], axis=1)

y_df = y_df.drop(['Department'], axis=1)
y_df.head()


,Attrition,Department_Human Resources,Department_Research & Development,Department_Sales
0,Yes,0.0,1.0,0.0
1,No,0.0,0.0,1.0
2,Yes,0.0,0.0,1.0
3,No,0.0,0.0,1.0
4,No,0.0,0.0,1.0


In [27]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
#attrition_encoded = attrition_encoder.fit(X_train_scaled, y_train)

# Create two new variables by applying the encoder
# to the training and testing data
attrition_training_encoded = attrition_encoder.fit_transform(y_train[['Attrition']])
attrition_testing_encoded = attrition_encoder.fit_transform(y_test[['Attrition']])
attrition_columns = attrition_encoder.get_feature_names_out(['Attrition'])
df_attrition = pd.DataFrame(attrition_training_encoded, columns=attrition_columns)
y_df = pd.concat([y_df, df_attrition], axis=1)

y_df = y_df.drop(['Attrition'], axis=1)
y_df.head()

,Department_Human Resources,Department_Research & Development,Department_Sales,Attrition_No,Attrition_Yes
0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,1.0,1.0,0.0
3,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,1.0,1.0,0.0


## Create, Compile, and Train the Model

In [18]:
# Find the number of columns in the X training data
num_columns = len(X_df.columns)

# Create the input layer
input_layer = layers.Input(shape=(num_columns,), name='input_features')


# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu')(shared_layer1)

In [19]:
# Create a branch for Department
# with a hidden layer and an output layer
 
# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)



In [20]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='softmax', name='attrition_output')(attrition_hidden)


In [21]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'quality_output': 'categorical_crossentropy', 'color_output': 'binary_crossentropy'},
              metrics={'quality_output': 'accuracy', 'color_output': 'accuracy'})


# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 64)        │        704 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 128)       │      8,320 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │      4,128 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      4,128 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_8[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ dense_9[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
X_train.head()


,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
1343,3,29,7,1,0,0,3,3,1,3
1121,4,36,1,3,0,0,3,1,0,6
1048,3,34,3,1,0,0,3,13,3,3
1393,3,27,9,4,0,0,3,7,0,1
527,3,32,10,4,0,0,2,10,0,1


In [22]:
# Train the model
model.fit(
    X_df,
    {'department_output': y_df['Department'], 'attrition_output': y_df['Attrition']},
    epochs=10,
    batch_size=32,
    validation_split=0.2
)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [17]:
# Evaluate the model with the testing data


12/12 [==============================] - 0s 3ms/step - loss: 4.1624 - department_output_loss: 3.1143 - attrition_output_loss: 1.0481 - department_output_accuracy: 0.5272 - attrition_output_accuracy: 0.8261


[4.162380218505859,
 3.114327907562256,
 1.0480519533157349,
 0.5271739363670349,
 0.8260869383811951]

In [18]:
# Print the accuracy for both department and attrition


Department predictions accuracy: 0.5271739363670349
Attrition predictions accuracy: 0.8260869383811951


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 